# Supersampling via SMOTE
__SMOTE:__ **S**ynthetic **M**inority **O**ver-sampling **Te**chnique
<br>
Datensatz: fraud_claims.csv

In [1]:
import pandas as pd
import numpy as np
import imblearn

In [2]:
fraud_df = pd.read_csv("fraud_claims.csv")
fraud_df

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17 00:00:00,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27 00:00:00,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06 00:00:00,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25 00:00:00,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06 00:00:00,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,941851,1991-07-16 00:00:00,OH,500/1000,1000,1310.80,0,431289,...,1,?,87200,17440,8720,61040,Honda,Accord,2006,N
996,285,41,186934,2014-01-05 00:00:00,IL,100/300,1000,1436.79,0,608177,...,3,?,108480,18080,18080,72320,Volkswagen,Passat,2015,N
997,130,34,918516,2003-02-17 00:00:00,OH,250/500,500,1383.49,3000000,442797,...,3,YES,67500,7500,7500,52500,Suburu,Impreza,1996,N
998,458,62,533940,2011-11-18 00:00:00,IL,500/1000,2000,1356.92,5000000,441714,...,1,YES,46980,5220,5220,36540,Audi,A5,1998,N


### Umwandlung des Merkmals `policy_state` in Numerische Werte

`OH` (Ohio) => 1
<br>
`IL` (Illinois) => 2
<br>
`IN` (Indiana) => 3

In [3]:
fraud_df["policy_state"] = fraud_df["policy_state"].replace("OH", 1)
fraud_df["policy_state"] = fraud_df["policy_state"].replace("IL", 2)
fraud_df["policy_state"] = fraud_df["policy_state"].replace("IN", 3)
fraud_df["police_report_available"] = fraud_df["police_report_available"].replace("YES", 1)
fraud_df["police_report_available"] = fraud_df["police_report_available"].replace("NO", 0)
fraud_df["police_report_available"] = fraud_df["police_report_available"].replace("?", np.nan)
fraud_df["insured_sex"] = fraud_df["insured_sex"].replace("MALE", 0)
fraud_df["insured_sex"] = fraud_df["insured_sex"].replace("FEMALE", 1)
fraud_df

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17 00:00:00,1,250/500,1000,1406.91,0,466132,...,2,1.0,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27 00:00:00,3,250/500,2000,1197.22,5000000,468176,...,0,NaN,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06 00:00:00,1,100/300,2000,1413.14,5000000,430632,...,3,0.0,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25 00:00:00,2,250/500,2000,1415.74,6000000,608117,...,2,0.0,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06 00:00:00,2,500/1000,1000,1583.91,6000000,610706,...,1,0.0,6500,1300,650,4550,Accura,RSX,2009,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,941851,1991-07-16 00:00:00,1,500/1000,1000,1310.80,0,431289,...,1,NaN,87200,17440,8720,61040,Honda,Accord,2006,N
996,285,41,186934,2014-01-05 00:00:00,2,100/300,1000,1436.79,0,608177,...,3,NaN,108480,18080,18080,72320,Volkswagen,Passat,2015,N
997,130,34,918516,2003-02-17 00:00:00,1,250/500,500,1383.49,3000000,442797,...,3,1.0,67500,7500,7500,52500,Suburu,Impreza,1996,N
998,458,62,533940,2011-11-18 00:00:00,2,500/1000,2000,1356.92,5000000,441714,...,1,1.0,46980,5220,5220,36540,Audi,A5,1998,N
